In [1]:
import math
import os
import json
import requests
import time

import systemlink.clients.nisysmgmt.api.systems_api as systems_api
from systemlink.clients.nisysmgmt.models import QuerySystemsRequest, ManageKeysRequest, KeyRequest, KeyAction

systems_handle = systems_api.SystemsApi(systems_api.ApiClient())

print("Getting the managed systems")
projection = 'new(id, workspace)'
query_systems_request = QuerySystemsRequest(projection = projection)
managed_systems = (await systems_handle.get_systems_by_query(query_systems_request)).data
print("Found {0} managed systems\n".format(len(managed_systems)))
# print(managed_systems)

print("Getting the systems keys")
systems_keys = await systems_handle.get_systems_keys()
print("Found {0} approved systems".format(len(systems_keys.systems_approved) if systems_keys.systems_approved else 0))
print("Found {0} pending systems".format(len(systems_keys.systems_pending) if systems_keys.systems_pending else 0))
print("Found {0} denied systems".format(len(systems_keys.systems_denied) if systems_keys.systems_denied else 0))
print("Found {0} rejected systems\n".format(len(systems_keys.systems_rejected) if systems_keys.systems_rejected else 0 ))
# print(systems_keys.systems_pending)

print("Approving the pending systems that are currently managed")
if systems_keys.systems_pending is None:
    systems_keys.systems_pending = []
managed_systems_to_approve = [system for system in managed_systems if system.get("id") in systems_keys.systems_pending]
print("Found {} managed systems that are in the pending state\n".format(len(managed_systems_to_approve)))
# managed_systems_to_approve = [{"id":"Precision_5570--SN-GPP8WT3--MAC-14-75-5B-DC-5F-AA","workspace":"846e294a-a007-47ac-9fc2-fac07eab240e"}]

batch_size = 50
batches = math.ceil(len(managed_systems_to_approve) / batch_size)
print("Approving {} batches of {} systems".format(batches, batch_size))

for i in range(0, batches):
    minions = managed_systems_to_approve[i * batch_size : (i+1) * batch_size]
    print("Approving {} minions, batch: {}".format(len(minions), i+1))
    keys_actions = [KeyRequest(minion.get("id"), KeyAction.ACCEPT, None, minion.get("workspace")) for minion in minions]
    print(keys_actions)
    manage_keys_request = ManageKeysRequest(is_async=True, key_actions = keys_actions)
    result = await systems_handle.manage_systems_keys(manage_keys_request)
    print("Manage Systems Keys Result: {}", result)
    time.sleep(10)



Getting the managed systems
Found 16 managed systems

Getting the systems keys
Found 6 approved systems
Found 38 pending systems
Found 0 denied systems
Found 0 rejected systems

Approving the pending systems that are currently managed
Found 2 managed systems that are in the pending state

Approving 1 batches of 50 systems
Approving 2 minions, batch: 1
[{'action': 'ACCEPT',
 'id': 'BTS-LabPC-1',
 'key': None,
 'workspace': 'ccae109b-38dc-4ba5-894b-93d0625326e2'}, {'action': 'ACCEPT',
 'id': 'Precision_5570--SN-GPP8WT3--MAC-14-75-5B-DC-5F-AA',
 'key': None,
 'workspace': 'ccae109b-38dc-4ba5-894b-93d0625326e2'}]
Manage Systems Keys Result: {} {'error': None}
